In [ ]:
import ipywidgets as widgets

from scipy.misc import imread, imsave
import io
import sqlite3

#

import debounce

path = "./test3/"

In [ ]:
def make_connection():
    return sqlite3.connect(path+'capture.sqlite')

In [ ]:
def query_users_info():
    conn = make_connection()
    c = conn.cursor()
    #c.execute('''SELECT min(student), max(student), count(distinct student)
    #             FROM capture_zone''')
    #stats = c.fetchall()
    c.execute('''SELECT DISTINCT student FROM capture_zone''')
    return [i[0] for i in c.fetchall()]

In [ ]:
def query_for_user(u):
    conn = make_connection()
    c = conn.cursor()
    c.execute('''SELECT id_a,*
                 FROM capture_zone
                 WHERE student=? AND type=?
                 ORDER BY id_a,id_b ASC
                 LIMIT 100
              ''',
#             ORDER BY black DESC
              (u, 4))

    boxes = c.fetchall()
    return boxes

def preload_all_queries():
    conn = make_connection()
    c = conn.cursor()
    c.execute('''SELECT id_a,student,* FROM capture_zone WHERE type=4 ORDER BY student,id_a,id_b ASC''')
    res = {}
    for r in c.fetchall():
        k = r[1]
        if not (k in res):
            res[k] = []
        res[k].append(r)
    return res

In [ ]:
cache = preload_all_queries()

In [ ]:
def flow_images_grouped_by(boxes, ind=0, maxline=24):
    res = []
    group = -123.123
    for b in boxes:
        if b[ind] != group:
            group = b[ind]
            res.append(widgets.HTML('''<div style="border: 1px solid black;"></div>'''+str(group)))
            res.append(widgets.HBox([]))
        if len(res) > 0 and len(res[-1].children) >= maxline:
            res.append(widgets.HBox([]))
        res[-1].children += (widgets.Image(value=b[-1], width=20, height=20),)
    return widgets.VBox(res)
#flow_images_grouped_by(query_for_user(16))

In [ ]:
all_users = query_users_info()
container = widgets.Box()

@debounce.debounce(.250)
def on_change_user(change):
    #container.children = (flow_images_grouped_by(query_for_user(all_users[change.new])), )
    container.children = (flow_images_grouped_by(cache[all_users[change.new]]), )

slider = widgets.IntSlider(min=1, max=len(all_users), value=1, description='User (in list)')
slider.observe(on_change_user, names='value')
display(slider, container)

In [ ]:
# PADDING


































